# Lesson 9b: Recurrent Neural Networks & Sequence Processing

**Learning Objectives:**
- Understand the fundamental architecture and mechanics of Recurrent Neural Networks (RNNs)
- Learn the limitations of vanilla RNNs and how LSTM and GRU architectures solve them
- Master sequence-to-sequence modeling for various tasks
- Implement bidirectional RNNs for improved context understanding
- Apply RNNs to time series forecasting and natural language processing
- Understand when to use RNNs versus Transformers in modern applications

**Prerequisites:**
- Understanding of basic neural networks (Lesson 3a, 3b)
- Familiarity with backpropagation and gradient descent
- Basic knowledge of CNNs (Lesson 9a) is helpful but not required

**Why RNNs Matter in 2025:**
While Transformers dominate many NLP tasks, RNNs remain critical for time series analysis, real-time streaming data, resource-constrained environments, and scenarios requiring sequential memory. Understanding RNNs provides essential foundations for comprehending modern sequence models and helps you choose the right architecture for specific constraints.

---

## 1. Installation & Setup

This notebook requires TensorFlow/Keras for implementing RNN architectures. We'll automatically install all required dependencies.

In [ ]:
# Auto-install required packages
import sys
import subprocess

def install_package(package_name):
    """Install a package using pip if not already installed."""
    try:
        __import__(package_name.split('[')[0])
        print(f"✓ {package_name} already installed")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package_name])
        print(f"✓ {package_name} installed successfully")

# Required packages
packages = [
    'tensorflow>=2.13.0',
    'numpy>=1.24.0',
    'matplotlib>=3.7.0',
    'scikit-learn>=1.3.0',
    'pandas>=2.0.0',
    'seaborn>=0.12.0'
]

for package in packages:
    install_package(package)

print("\n✓ All dependencies installed successfully!")

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List
import warnings

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Scikit-learn utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

---

## 2. RNN Fundamentals: Understanding Sequential Processing

### 2.1 Why Do We Need RNNs?

Traditional neural networks (feedforward, CNNs) have a critical limitation: they cannot handle **variable-length sequences** or maintain **temporal dependencies**. They treat each input independently.

**Examples where this fails:**
- Language: "The cat sat on the mat" vs "The cats sat on the mat" (different lengths)
- Time series: Stock prices where past values influence future predictions
- Video: Frame sequences where temporal order matters

### 2.2 RNN Core Concept

RNNs solve this by maintaining **hidden state** (memory) that gets updated at each time step:

```
h_t = tanh(W_hh * h_{t-1} + W_xh * x_t + b_h)
y_t = W_hy * h_t + b_y
```

Where:
- `h_t`: Hidden state at time t (memory)
- `x_t`: Input at time t
- `y_t`: Output at time t
- `W_*`: Weight matrices (learned parameters)

**Key innovation**: The hidden state `h_t` depends on both the current input `x_t` AND the previous hidden state `h_{t-1}`, creating a memory chain.

Let's visualize this:

In [ ]:
# Implement a simple vanilla RNN from scratch for educational purposes

class SimpleRNN:
    """Simple RNN implementation from scratch for demonstration."""
    
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        
        # Initialize weights randomly (small values)
        self.W_xh = np.random.randn(hidden_size, input_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(output_size, hidden_size) * 0.01
        
        # Initialize biases
        self.b_h = np.zeros((hidden_size, 1))
        self.b_y = np.zeros((output_size, 1))
    
    def forward(self, inputs):
        """
        Forward pass through RNN.
        
        Args:
            inputs: List of input vectors, one per time step
        
        Returns:
            outputs: List of output vectors
            hidden_states: List of hidden states (for visualization)
        """
        h = np.zeros((self.hidden_size, 1))  # Initial hidden state
        
        outputs = []
        hidden_states = [h.copy()]
        
        for x in inputs:
            # Reshape input to column vector
            x = x.reshape(-1, 1)
            
            # Update hidden state
            # h_t = tanh(W_hh * h_{t-1} + W_xh * x_t + b_h)
            h = np.tanh(np.dot(self.W_hh, h) + np.dot(self.W_xh, x) + self.b_h)
            
            # Compute output
            # y_t = W_hy * h_t + b_y
            y = np.dot(self.W_hy, h) + self.b_y
            
            outputs.append(y)
            hidden_states.append(h.copy())
        
        return outputs, hidden_states

# Demonstrate RNN on a simple sequence
print("Demonstrating Vanilla RNN on Simple Sequence:\n")
print("="*60)

# Create a simple RNN
rnn = SimpleRNN(input_size=3, hidden_size=5, output_size=2)

# Create a simple input sequence (4 time steps, 3 features each)
input_sequence = [
    np.array([1.0, 0.0, 0.0]),
    np.array([0.0, 1.0, 0.0]),
    np.array([0.0, 0.0, 1.0]),
    np.array([1.0, 1.0, 0.0])
]

# Forward pass
outputs, hidden_states = rnn.forward(input_sequence)

print(f"Input sequence length: {len(input_sequence)} time steps")
print(f"Input dimension: {input_sequence[0].shape[0]} features")
print(f"Hidden state dimension: {rnn.hidden_size}")
print(f"Output dimension: {outputs[0].shape[0]}")
print("\nKey Insight: The hidden state acts as memory, carrying information")
print("from previous time steps to influence current and future predictions.")
print("="*60)

# Visualize hidden state evolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot hidden states over time
hidden_array = np.hstack(hidden_states).T
im1 = ax1.imshow(hidden_array, cmap='RdBu', aspect='auto')
ax1.set_xlabel('Hidden Units', fontsize=11)
ax1.set_ylabel('Time Step', fontsize=11)
ax1.set_title('Hidden State Evolution Over Time', fontsize=12, fontweight='bold')
plt.colorbar(im1, ax=ax1)

# Plot output evolution
output_array = np.hstack(outputs).T
ax2.plot(output_array[:, 0], marker='o', label='Output Dim 1', linewidth=2)
ax2.plot(output_array[:, 1], marker='s', label='Output Dim 2', linewidth=2)
ax2.set_xlabel('Time Step', fontsize=11)
ax2.set_ylabel('Output Value', fontsize=11)
ax2.set_title('RNN Output Over Time', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('rnn_hidden_state_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

### 2.3 The Vanishing Gradient Problem

**Critical limitation of vanilla RNNs:**

During backpropagation through time (BPTT), gradients are multiplied repeatedly by the weight matrix `W_hh`. This causes:

1. **Vanishing gradients**: If eigenvalues of `W_hh` < 1, gradients shrink exponentially → RNN cannot learn long-term dependencies
2. **Exploding gradients**: If eigenvalues of `W_hh` > 1, gradients grow exponentially → training becomes unstable

**Practical impact**: Vanilla RNNs struggle to remember information beyond 10-20 time steps.

**Solution**: LSTM and GRU architectures with gating mechanisms!

---

## 3. LSTM: Long Short-Term Memory Networks

### 3.1 LSTM Architecture

LSTMs solve the vanishing gradient problem through a sophisticated **gating mechanism** that controls information flow:

**Three gates:**
1. **Forget gate** (`f_t`): Decides what information to discard from cell state
2. **Input gate** (`i_t`): Decides what new information to store in cell state
3. **Output gate** (`o_t`): Decides what information to output from cell state

**Key equations:**
```
f_t = σ(W_f · [h_{t-1}, x_t] + b_f)      # Forget gate
i_t = σ(W_i · [h_{t-1}, x_t] + b_i)      # Input gate
C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)  # Candidate values
C_t = f_t * C_{t-1} + i_t * C̃_t          # Update cell state
o_t = σ(W_o · [h_{t-1}, x_t] + b_o)      # Output gate
h_t = o_t * tanh(C_t)                    # Hidden state output
```

Where `σ` is sigmoid function (outputs 0-1, perfect for gating).

**Genius of LSTM**: The cell state `C_t` can flow through time with minimal modification, allowing gradients to flow more easily during backpropagation.

Let's implement an LSTM for sequence prediction:

In [ ]:
# Generate synthetic time series data for demonstration

def generate_sine_wave_data(n_samples=1000, sequence_length=50):
    """
    Generate synthetic sine wave data for time series prediction.
    
    Args:
        n_samples: Total number of time points
        sequence_length: Length of each input sequence
    
    Returns:
        X: Input sequences of shape (num_sequences, sequence_length, 1)
        y: Target values (next value in sequence)
    """
    # Generate sine wave with some noise
    time = np.linspace(0, 100, n_samples)
    signal = np.sin(time) + 0.5 * np.sin(3 * time) + 0.1 * np.random.randn(n_samples)
    
    # Create sequences
    X, y = [], []
    for i in range(len(signal) - sequence_length):
        X.append(signal[i:i + sequence_length])
        y.append(signal[i + sequence_length])
    
    X = np.array(X).reshape(-1, sequence_length, 1)
    y = np.array(y)
    
    return X, y, signal

# Generate data
print("Generating synthetic time series data...\n")
X_seq, y_seq, full_signal = generate_sine_wave_data(n_samples=1000, sequence_length=50)

# Split into train/val/test
train_size = int(0.7 * len(X_seq))
val_size = int(0.15 * len(X_seq))

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]

X_val = X_seq[train_size:train_size + val_size]
y_val = y_seq[train_size:train_size + val_size]

X_test = X_seq[train_size + val_size:]
y_test = y_seq[train_size + val_size:]

print(f"Data splits:")
print(f"Train: {X_train.shape}, Target: {y_train.shape}")
print(f"Validation: {X_val.shape}, Target: {y_val.shape}")
print(f"Test: {X_test.shape}, Target: {y_test.shape}")

# Visualize the time series
plt.figure(figsize=(14, 5))
plt.plot(full_signal[:300], linewidth=1.5, alpha=0.8)
plt.axvline(x=train_size, color='red', linestyle='--', label='Train/Val Split', linewidth=2)
plt.axvline(x=train_size + val_size, color='orange', linestyle='--', 
           label='Val/Test Split', linewidth=2)
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.title('Synthetic Time Series Data (First 300 Points)', fontsize=13, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('time_series_data.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Build LSTM model for time series forecasting

def build_lstm_model(sequence_length=50, lstm_units=64, num_layers=2):
    """
    Build LSTM model for time series prediction.
    
    Args:
        sequence_length: Length of input sequences
        lstm_units: Number of LSTM units per layer
        num_layers: Number of stacked LSTM layers
    
    Returns:
        Compiled Keras model
    """
    model = models.Sequential()
    
    # First LSTM layer (return sequences for stacking)
    model.add(layers.LSTM(
        lstm_units,
        return_sequences=(num_layers > 1),
        input_shape=(sequence_length, 1),
        name='lstm_1'
    ))
    model.add(layers.Dropout(0.2))
    
    # Additional LSTM layers
    for i in range(1, num_layers):
        return_seq = (i < num_layers - 1)
        model.add(layers.LSTM(
            lstm_units,
            return_sequences=return_seq,
            name=f'lstm_{i+1}'
        ))
        model.add(layers.Dropout(0.2))
    
    # Output layer
    model.add(layers.Dense(32, activation='relu', name='dense_1'))
    model.add(layers.Dense(1, name='output'))
    
    # Compile
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Build and display model
lstm_model = build_lstm_model(sequence_length=50, lstm_units=64, num_layers=2)

print("\n" + "="*70)
print("LSTM MODEL ARCHITECTURE")
print("="*70)
lstm_model.summary()
print("="*70)

In [ ]:
# Train LSTM model

print("\nTraining LSTM model...\n")

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

history_lstm = lstm_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n✓ LSTM training completed!")

In [ ]:
# Evaluate LSTM performance

# Make predictions
y_pred_train = lstm_model.predict(X_train, verbose=0).flatten()
y_pred_val = lstm_model.predict(X_val, verbose=0).flatten()
y_pred_test = lstm_model.predict(X_test, verbose=0).flatten()

# Calculate metrics
train_mse = mean_squared_error(y_train, y_pred_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)

test_mse = mean_squared_error(y_test, y_pred_test)
test_mae = mean_absolute_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)

print("\n" + "="*60)
print("LSTM MODEL PERFORMANCE")
print("="*60)
print(f"\nTraining Set:")
print(f"  MSE: {train_mse:.6f}")
print(f"  MAE: {train_mae:.6f}")
print(f"  R²:  {train_r2:.6f}")
print(f"\nTest Set:")
print(f"  MSE: {test_mse:.6f}")
print(f"  MAE: {test_mae:.6f}")
print(f"  R²:  {test_r2:.6f}")
print("="*60)

# Visualize predictions
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Training history
axes[0].plot(history_lstm.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history_lstm.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss (MSE)', fontsize=11)
axes[0].set_title('LSTM Training History', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Predictions vs actual
test_range = range(len(y_test))
axes[1].plot(test_range, y_test, label='Actual', linewidth=2, alpha=0.7)
axes[1].plot(test_range, y_pred_test, label='LSTM Predictions', 
            linewidth=2, alpha=0.7, linestyle='--')
axes[1].set_xlabel('Time Step', fontsize=11)
axes[1].set_ylabel('Value', fontsize=11)
axes[1].set_title('LSTM Predictions vs Actual (Test Set)', fontsize=12, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_performance.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 4. GRU: Gated Recurrent Units

### 4.1 GRU Architecture

GRUs are a simpler alternative to LSTMs, introduced in 2014. They combine the forget and input gates into a single **update gate** and merge the cell state and hidden state.

**Two gates instead of three:**
1. **Update gate** (`z_t`): Decides how much past information to keep
2. **Reset gate** (`r_t`): Decides how much past information to forget

**Key equations:**
```
z_t = σ(W_z · [h_{t-1}, x_t])           # Update gate
r_t = σ(W_r · [h_{t-1}, x_t])           # Reset gate
h̃_t = tanh(W · [r_t * h_{t-1}, x_t])   # Candidate activation
h_t = (1 - z_t) * h_{t-1} + z_t * h̃_t  # Final hidden state
```

**GRU vs LSTM:**
- **GRU advantages**: Fewer parameters (~25% less), faster training, works well on smaller datasets
- **LSTM advantages**: More expressive, better on complex tasks with large datasets
- **In practice (2025)**: Try both! GRU often performs similarly with less computation.

Let's compare GRU vs LSTM:

In [ ]:
# Build GRU model for comparison

def build_gru_model(sequence_length=50, gru_units=64, num_layers=2):
    """
    Build GRU model for time series prediction.
    
    Args:
        sequence_length: Length of input sequences
        gru_units: Number of GRU units per layer
        num_layers: Number of stacked GRU layers
    
    Returns:
        Compiled Keras model
    """
    model = models.Sequential()
    
    # First GRU layer
    model.add(layers.GRU(
        gru_units,
        return_sequences=(num_layers > 1),
        input_shape=(sequence_length, 1),
        name='gru_1'
    ))
    model.add(layers.Dropout(0.2))
    
    # Additional GRU layers
    for i in range(1, num_layers):
        return_seq = (i < num_layers - 1)
        model.add(layers.GRU(
            gru_units,
            return_sequences=return_seq,
            name=f'gru_{i+1}'
        ))
        model.add(layers.Dropout(0.2))
    
    # Output layer
    model.add(layers.Dense(32, activation='relu', name='dense_1'))
    model.add(layers.Dense(1, name='output'))
    
    # Compile
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Build GRU model
gru_model = build_gru_model(sequence_length=50, gru_units=64, num_layers=2)

print("\n" + "="*70)
print("GRU MODEL ARCHITECTURE")
print("="*70)
gru_model.summary()
print("="*70)

# Compare parameter counts
lstm_params = lstm_model.count_params()
gru_params = gru_model.count_params()

print(f"\nParameter Comparison:")
print(f"LSTM parameters: {lstm_params:,}")
print(f"GRU parameters:  {gru_params:,}")
print(f"Reduction:       {((lstm_params - gru_params) / lstm_params * 100):.1f}%")

In [ ]:
# Train GRU model

print("\nTraining GRU model...\n")

history_gru = gru_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("\n✓ GRU training completed!")

# Evaluate GRU
y_pred_gru = gru_model.predict(X_test, verbose=0).flatten()
gru_mse = mean_squared_error(y_test, y_pred_gru)
gru_mae = mean_absolute_error(y_test, y_pred_gru)
gru_r2 = r2_score(y_test, y_pred_gru)

# Compare LSTM vs GRU
print("\n" + "="*70)
print("LSTM vs GRU COMPARISON (Test Set)")
print("="*70)
print(f"{'Metric':<15} {'LSTM':<20} {'GRU':<20}")
print("="*70)
print(f"{'MSE':<15} {test_mse:<20.6f} {gru_mse:<20.6f}")
print(f"{'MAE':<15} {test_mae:<20.6f} {gru_mae:<20.6f}")
print(f"{'R²':<15} {test_r2:<20.6f} {gru_r2:<20.6f}")
print(f"{'Parameters':<15} {lstm_params:<20,} {gru_params:<20,}")
print("="*70)

---

## 5. Bidirectional RNNs: Context from Both Directions

### 5.1 The Bidirectional Concept

Standard RNNs process sequences in one direction (left-to-right). But for many tasks, **future context is also valuable**:

**Example**: "The animal didn't cross the street because it was too **tired**"
- Understanding "it" requires context from BOTH before ("animal") and after ("tired")

**Bidirectional RNNs** solve this by:
1. Running one RNN forward (left-to-right)
2. Running another RNN backward (right-to-left)
3. Concatenating both hidden states at each time step

**When to use bidirectional:**
- ✅ Text classification, named entity recognition, sentiment analysis
- ✅ Speech recognition
- ✅ Any task where full sequence is available at inference
- ❌ Real-time prediction (no future context available)
- ❌ Time series forecasting (cannot see the future!)

Let's demonstrate with sentiment analysis:

In [ ]:
# Create synthetic sentiment analysis dataset

# Sample movie reviews (simplified for demonstration)
positive_reviews = [
    "This movie was absolutely amazing and wonderful",
    "I loved every minute of this film",
    "Outstanding performance by all actors",
    "A masterpiece of modern cinema",
    "Brilliant storytelling and direction",
    "Highly recommend this fantastic movie",
    "One of the best films I have ever seen",
    "Exceptional quality and entertainment",
    "Thoroughly enjoyed this incredible film",
    "Amazing visuals and great plot"
] * 10  # Repeat for more samples

negative_reviews = [
    "This movie was terrible and boring",
    "I hated every minute of this film",
    "Poor performance by all actors",
    "A disaster of modern cinema",
    "Awful storytelling and direction",
    "Do not recommend this horrible movie",
    "One of the worst films I have ever seen",
    "Terrible quality and no entertainment",
    "Could not stand this dreadful film",
    "Bad visuals and weak plot"
] * 10  # Repeat for more samples

# Combine and create labels
all_reviews = positive_reviews + negative_reviews
all_labels = [1] * len(positive_reviews) + [0] * len(negative_reviews)

# Shuffle
indices = np.random.permutation(len(all_reviews))
all_reviews = [all_reviews[i] for i in indices]
all_labels = [all_labels[i] for i in indices]

print(f"Total reviews: {len(all_reviews)}")
print(f"Positive: {sum(all_labels)}, Negative: {len(all_labels) - sum(all_labels)}")
print("\nSample reviews:")
for i in range(3):
    label = "Positive" if all_labels[i] == 1 else "Negative"
    print(f"  [{label}] {all_reviews[i]}")

In [ ]:
# Tokenize and prepare sequences

# Create tokenizer
max_words = 1000
max_len = 20

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(all_reviews)

# Convert to sequences
sequences = tokenizer.texts_to_sequences(all_reviews)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Convert labels to array
labels_array = np.array(all_labels)

# Train/val/test split
X_temp, X_test_nlp, y_temp, y_test_nlp = train_test_split(
    padded_sequences, labels_array, test_size=0.2, random_state=42, stratify=labels_array
)
X_train_nlp, X_val_nlp, y_train_nlp, y_val_nlp = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print(f"\nNLP Dataset splits:")
print(f"Train: {X_train_nlp.shape}")
print(f"Validation: {X_val_nlp.shape}")
print(f"Test: {X_test_nlp.shape}")
print(f"Vocabulary size: {len(tokenizer.word_index)}")

In [ ]:
# Build Bidirectional LSTM model for sentiment analysis

def build_bidirectional_lstm(vocab_size, embedding_dim=64, lstm_units=64):
    """
    Build bidirectional LSTM for text classification.
    
    Args:
        vocab_size: Size of vocabulary
        embedding_dim: Dimension of word embeddings
        lstm_units: Number of LSTM units
    
    Returns:
        Compiled Keras model
    """
    model = models.Sequential([
        # Embedding layer: converts word indices to dense vectors
        layers.Embedding(vocab_size, embedding_dim, name='embedding'),
        
        # Bidirectional LSTM: processes sequence in both directions
        layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True), 
                           name='bidirectional_lstm_1'),
        layers.Dropout(0.3),
        
        # Second bidirectional layer
        layers.Bidirectional(layers.LSTM(lstm_units), name='bidirectional_lstm_2'),
        layers.Dropout(0.3),
        
        # Output layer
        layers.Dense(32, activation='relu', name='dense'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid', name='output')
    ])
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build bidirectional model
bi_lstm = build_bidirectional_lstm(vocab_size=max_words, embedding_dim=64, lstm_units=64)

print("\n" + "="*70)
print("BIDIRECTIONAL LSTM MODEL")
print("="*70)
bi_lstm.summary()
print("="*70)
print("\nKey: Bidirectional layers process sequences in BOTH directions,")
print("     doubling the hidden state size (forward + backward).")

In [ ]:
# Train bidirectional LSTM

print("\nTraining Bidirectional LSTM...\n")

history_bi = bi_lstm.fit(
    X_train_nlp, y_train_nlp,
    batch_size=32,
    epochs=30,
    validation_data=(X_val_nlp, y_val_nlp),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    ],
    verbose=1
)

# Evaluate
test_loss, test_acc = bi_lstm.evaluate(X_test_nlp, y_test_nlp, verbose=0)

print("\n" + "="*60)
print("BIDIRECTIONAL LSTM RESULTS")
print("="*60)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print("="*60)

# Visualize training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history_bi.history['accuracy'], label='Training', linewidth=2)
ax1.plot(history_bi.history['val_accuracy'], label='Validation', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=11)
ax1.set_ylabel('Accuracy', fontsize=11)
ax1.set_title('Bidirectional LSTM Accuracy', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history_bi.history['loss'], label='Training', linewidth=2)
ax2.plot(history_bi.history['val_loss'], label='Validation', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=11)
ax2.set_ylabel('Loss', fontsize=11)
ax2.set_title('Bidirectional LSTM Loss', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('bidirectional_lstm_training.png', dpi=150, bbox_inches='tight')
plt.show()

# Test on custom examples
test_sentences = [
    "This movie was absolutely amazing",
    "I hated this terrible film",
    "Not bad but could be better"
]

test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs, maxlen=max_len, padding='post')
predictions = bi_lstm.predict(test_padded, verbose=0)

print("\nCustom Predictions:")
for sent, pred in zip(test_sentences, predictions):
    sentiment = "Positive" if pred[0] > 0.5 else "Negative"
    print(f"  '{sent}'")
    print(f"  → {sentiment} (confidence: {pred[0]:.2f})\n")

---

## 6. Sequence-to-Sequence Models

### 6.1 Encoder-Decoder Architecture

Many tasks require mapping one sequence to another sequence of **different length**:
- Machine translation: "Hello" → "Bonjour"
- Text summarization: Long article → Short summary
- Question answering: Question + Context → Answer

**Sequence-to-Sequence (Seq2Seq) architecture:**

1. **Encoder**: Processes input sequence, compresses into context vector (final hidden state)
2. **Decoder**: Generates output sequence from context vector

**Example flow:**
```
Input:  "How are you?"
Encoder: [h1, h2, h3, h4] → context_vector = h4
Decoder: context_vector → ["Comment", "allez", "vous", "?"]
Output: "Comment allez vous?"
```

### 6.2 Attention Mechanism (Preview)

**Problem with basic seq2seq**: The context vector is a bottleneck - it must encode the entire input sequence!

**Attention mechanism** (introduced 2015) solves this by:
- Letting decoder "attend" to different parts of input at each decoding step
- Computing weighted combination of all encoder hidden states

**Impact**: Attention revolutionized NLP and led directly to Transformers (covered in Lesson 9c)!

We'll demonstrate a simple seq2seq model for illustration:

In [ ]:
# Simple sequence-to-sequence example: Reversing sequences

def generate_reverse_sequences(num_samples=1000, seq_length=10, vocab_size=20):
    """
    Generate sequences and their reverses for seq2seq training.
    
    Task: Learn to reverse a sequence of integers.
    Example: [3, 7, 2, 9] → [9, 2, 7, 3]
    """
    X = np.random.randint(1, vocab_size, size=(num_samples, seq_length))
    y = np.flip(X, axis=1)
    
    return X, y

# Generate data
seq_X, seq_y = generate_reverse_sequences(num_samples=5000, seq_length=10, vocab_size=20)

# Split
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    seq_X, seq_y, test_size=0.2, random_state=42
)

print("Sequence Reversal Task:")
print(f"Training samples: {len(X_train_seq)}")
print(f"Test samples: {len(X_test_seq)}")
print(f"Sequence length: {seq_X.shape[1]}")
print(f"Vocabulary size: 20\n")

print("Example sequences:")
for i in range(3):
    print(f"  Input:  {X_train_seq[i].tolist()}")
    print(f"  Target: {y_train_seq[i].tolist()}\n")

In [ ]:
# Build simple seq2seq model

def build_seq2seq_model(vocab_size=20, seq_length=10, embedding_dim=32, lstm_units=64):
    """
    Build simple sequence-to-sequence model.
    
    Note: This is a simplified version for demonstration.
    Production seq2seq uses encoder-decoder with separate models.
    """
    model = models.Sequential([
        layers.Embedding(vocab_size + 1, embedding_dim, input_length=seq_length),
        layers.LSTM(lstm_units, return_sequences=True),
        layers.LSTM(lstm_units, return_sequences=True),
        layers.TimeDistributed(layers.Dense(vocab_size + 1, activation='softmax'))
    ])
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

seq2seq_model = build_seq2seq_model()

print("\n" + "="*70)
print("SEQUENCE-TO-SEQUENCE MODEL")
print("="*70)
seq2seq_model.summary()
print("="*70)

# Train
print("\nTraining seq2seq model...\n")

history_seq2seq = seq2seq_model.fit(
    X_train_seq, y_train_seq,
    batch_size=64,
    epochs=30,
    validation_split=0.2,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

# Evaluate
test_loss, test_acc = seq2seq_model.evaluate(X_test_seq, y_test_seq, verbose=0)

print(f"\n{'='*60}")
print(f"Seq2Seq Test Accuracy: {test_acc*100:.2f}%")
print(f"{'='*60}")

# Test predictions
predictions = seq2seq_model.predict(X_test_seq[:5], verbose=0)
pred_sequences = np.argmax(predictions, axis=-1)

print("\nSample Predictions:")
for i in range(5):
    print(f"  Input:     {X_test_seq[i].tolist()}")
    print(f"  Predicted: {pred_sequences[i].tolist()}")
    print(f"  True:      {y_test_seq[i].tolist()}")
    print(f"  Match:     {np.array_equal(pred_sequences[i], y_test_seq[i])}\n")

---

## 7. Production Best Practices for RNNs

### 7.1 When to Use RNNs (2025 Guidance)

**✅ Use RNNs/LSTMs/GRUs when:**
- Working with time series data (stock prices, sensor data, energy consumption)
- Resource constraints make Transformers impractical
- Sequential processing is needed in real-time streaming scenarios
- Sequence length is moderate (< 500 tokens)
- You need online/incremental learning

**⚠️ Consider Transformers instead when:**
- Working with long sequences (> 500 tokens)
- Parallelization is critical for training speed
- You have large datasets and computational resources
- Working on NLP tasks (text classification, translation, QA)

### 7.2 Training Best Practices

**Gradient Clipping:**
Essential for preventing exploding gradients in RNNs:
```python
optimizer = tf.keras.optimizers.Adam(clipnorm=1.0)  # Clip by norm
# or
optimizer = tf.keras.optimizers.Adam(clipvalue=0.5)  # Clip by value
```

**Sequence Padding:**
- Pad sequences to same length for batch processing
- Use `padding='post'` for better performance
- Use masking to ignore padded values

**Stateful RNNs:**
- For very long sequences, use stateful RNNs to maintain state across batches
- Manually reset states between independent sequences

### 7.3 Optimization Techniques

In [ ]:
# Production-ready RNN pipeline

class RNNProductionPipeline:
    """
    Production-ready RNN training pipeline with best practices.
    """
    
    def __init__(self, rnn_type='LSTM'):
        self.rnn_type = rnn_type
        self.model = None
        self.scaler = MinMaxScaler()
    
    def build_model(self, input_shape, output_dim, units=64, num_layers=2):
        """
        Build RNN model with best practices.
        
        Best practices included:
        - Layer normalization
        - Gradient clipping
        - Dropout regularization
        - Proper activation functions
        """
        model = models.Sequential()
        
        # Choose RNN type
        RNN_Layer = layers.LSTM if self.rnn_type == 'LSTM' else layers.GRU
        
        # Build stacked RNN
        for i in range(num_layers):
            return_seq = (i < num_layers - 1)
            
            if i == 0:
                model.add(RNN_Layer(
                    units,
                    return_sequences=return_seq,
                    input_shape=input_shape,
                    recurrent_dropout=0.2,  # Recurrent dropout
                    name=f'{self.rnn_type.lower()}_{i+1}'
                ))
            else:
                model.add(RNN_Layer(
                    units,
                    return_sequences=return_seq,
                    recurrent_dropout=0.2,
                    name=f'{self.rnn_type.lower()}_{i+1}'
                ))
            
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(0.3))
        
        # Output layers
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dropout(0.2))
        model.add(layers.Dense(output_dim))
        
        # Compile with gradient clipping
        model.compile(
            optimizer=optimizers.Adam(learning_rate=0.001, clipnorm=1.0),  # Gradient clipping
            loss='mse',
            metrics=['mae']
        )
        
        self.model = model
        return model
    
    def get_callbacks(self):
        """Production callbacks."""
        return [
            EarlyStopping(
                monitor='val_loss',
                patience=15,
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=7,
                min_lr=1e-7,
                verbose=1
            ),
            ModelCheckpoint(
                'best_rnn_model.h5',
                monitor='val_loss',
                save_best_only=True,
                verbose=1
            )
        ]
    
    def prepare_sequences(self, data, sequence_length, target_column=-1):
        """
        Prepare sequences from time series data.
        
        Args:
            data: Time series data (1D or 2D array)
            sequence_length: Length of input sequences
            target_column: Which column to predict (-1 for last)
        
        Returns:
            X, y: Prepared sequences
        """
        # Normalize data
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        
        data_normalized = self.scaler.fit_transform(data)
        
        # Create sequences
        X, y = [], []
        for i in range(len(data_normalized) - sequence_length):
            X.append(data_normalized[i:i + sequence_length])
            y.append(data_normalized[i + sequence_length, target_column])
        
        return np.array(X), np.array(y)
    
    def train(self, X_train, y_train, X_val, y_val, epochs=100, batch_size=32):
        """Train with all best practices."""
        
        history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=self.get_callbacks(),
            verbose=1
        )
        
        return history

# Demonstrate production pipeline
print("\nProduction RNN Pipeline:")
print("="*60)

pipeline = RNNProductionPipeline(rnn_type='LSTM')
model_prod = pipeline.build_model(input_shape=(50, 1), output_dim=1, units=64, num_layers=2)

print("\n✓ Production pipeline initialized with:")
print("  • Gradient clipping (clipnorm=1.0)")
print("  • Batch normalization")
print("  • Dropout regularization")
print("  • Recurrent dropout")
print("  • Early stopping")
print("  • Learning rate scheduling")
print("  • Model checkpointing")
print("  • Data normalization")

---

## 8. Summary & Key Takeaways

### What We Learned:

**RNN Fundamentals:**
- RNNs maintain hidden state (memory) to process sequences
- Vanilla RNNs suffer from vanishing/exploding gradients
- Cannot effectively learn dependencies beyond 10-20 time steps

**LSTM & GRU:**
- LSTM uses three gates (forget, input, output) to control information flow
- GRU simplifies to two gates (update, reset) with comparable performance
- Both solve vanishing gradient problem and handle long-term dependencies
- GRU has fewer parameters (~25% less) and trains faster

**Bidirectional RNNs:**
- Process sequences in both forward and backward directions
- Essential for tasks where full sequence context is available
- Double the hidden state size (forward + backward concatenated)
- Not suitable for real-time prediction or time series forecasting

**Sequence-to-Sequence:**
- Encoder-decoder architecture for variable-length input/output
- Attention mechanism revolutionized seq2seq models
- Foundation for modern Transformers (Lesson 9c)

**Production Best Practices:**
- Always use gradient clipping to prevent exploding gradients
- Normalize/scale input data for stable training
- Apply dropout and recurrent dropout for regularization
- Use proper sequence padding and masking
- Choose LSTM for complex tasks, GRU for efficiency

### RNNs vs Transformers (2025):

**Use RNNs when:**
- Time series forecasting
- Streaming/online learning
- Resource-constrained environments
- Moderate sequence lengths (< 500)

**Use Transformers when:**
- NLP tasks (classification, translation, QA)
- Long sequences (> 500 tokens)
- Large datasets and compute available
- Parallelization is critical

### Next Steps:

- **Lesson 9c**: Transformers & Attention - Master the architecture dominating AI in 2025
- **Advanced RNN topics**: Attention mechanisms, teacher forcing, beam search
- **Applications**: Speech recognition, video analysis, anomaly detection

---

## 9. Exercises & Further Exploration

### Exercise 1: Stock Price Prediction
Download real stock price data and build an LSTM model to predict future prices:
- Use multiple features (open, high, low, close, volume)
- Experiment with different sequence lengths
- Compare LSTM vs GRU performance

### Exercise 2: Text Generation
Train a character-level LSTM to generate text:
- Use a corpus (Shakespeare, news articles, code)
- Implement temperature sampling for diversity
- Visualize what the model learns at different layers

### Exercise 3: Bidirectional vs Unidirectional
Compare bidirectional and unidirectional models:
- Use the same sentiment analysis task
- Measure accuracy difference
- Analyze which performs better and why

### Further Reading:

- **Original Papers**:
  - LSTM: "Long Short-Term Memory" (Hochreiter & Schmidhuber, 1997)
  - GRU: "Learning Phrase Representations" (Cho et al., 2014)
  - Seq2Seq: "Sequence to Sequence Learning" (Sutskever et al., 2014)
  - Attention: "Neural Machine Translation by Jointly Learning to Align" (Bahdanau et al., 2015)

- **Resources**:
  - Stanford CS224n: Natural Language Processing with Deep Learning
  - "Understanding LSTM Networks" by Christopher Olah
  - TensorFlow Time Series Tutorial

---

**Congratulations!** You now understand RNNs, LSTMs, GRUs, and sequence modeling. These architectures remain essential for time series and streaming applications in 2025.

Continue to **Lesson 9c: Transformers & Attention** to learn the architecture that revolutionized AI! 🚀